In [3]:
# Load in PyTorch's pretrained network
import numpy as np
import os
import torchvision.models as models
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn.functional as func
import pandas as pd
from torch.utils.data.dataset import Dataset
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
from random import shuffle
import matplotlib.pyplot as plt
import sklearn.metrics as sk_metrics
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter


import matplotlib.pyplot as plt


from tqdm import tqdm

In [7]:

class chxrayData(Dataset):
    def __init__(self, dataframe,is_trainset):
        self.dataframe = dataframe
        classes = ['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia',
                   'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax',
                   'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema',
                   'Consolidation']

        self.labels = np.asarray(self.dataframe.loc[:,classes])
        self.img_names = self.dataframe['Image index']
       

        # Normalization and data augmentation
        self.train_transforms = transforms.Compose([
            transforms.Resize(270),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            # Normalize by using pre-computed dataset statistics
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
        self.test_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
        self.is_train = is_trainset
        

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        
        imagePath=self.img_names[index]
        if self.is_train:
            image = self.train_transforms(Image.open(imagePath).convert('RGB'))
            
        else:
            image = self.test_transforms(Image.open(imagePath).convert('RGB'))
        
        label = self.labels[index]
        
        return (image, label)



In [10]:
writer = SummaryWriter()

test_df=pd.read_csv('../input/data1123/test_list.csv')
train_df=pd.read_csv('../input/data1123/train_list.csv')
val_df=pd.read_csv('../input/data1123/val_list.csv')
# Create dataset and data loader
test_dataset =  chxrayData(test_df, False)
train_dataset =  chxrayData(train_df, True)
val_dataset =   chxrayData(val_df, False)

# More num_workers consumes more memory for good for speeding up I/O
# pin_memory=True enables fast data transfer to GPUs Source :https://www.kaggle.com/c/understanding_cloud_organization/discussion/112582
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, pin_memory=True, num_workers=8)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=True, pin_memory=True, num_workers=8)

In [11]:
# Source: https://www.kaggle.com/c/tgs-salt-identification-challenge/discussion/65938
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=1.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, model_output, target):
        BCE_loss = func.binary_cross_entropy_with_logits(model_output, target)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss

In [12]:

class CHXModel():
    
    
    def __init__(self):
        self.model = None
        self.losses = None
        self.optimizer = None
        self.model_losses = None
    # Freeze or un-freeze model layers as required
    def update_grad(self, grad_val):
        for param in self.model.parameters():
            param.requires_grad = grad_val
    
     # Initial model setup
    def set_up_model(self, n_classes, lr, unfreeze, pretrained, use_focal, gamma=1.0, alpha=0.25):
        self.model = models.resnet34(pretrained=pretrained, progress=True)
        # Freeze all layers
        self.update_grad(unfreeze)
        # Resnet has one fully connected layer, which outputs dimensions of n_classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = torch.nn.Linear(num_ftrs, n_classes)
        self.model.cuda()
        
        ''' Use a binary cross-entropy loss function; Applies sigmoid internally (generating probabilities)
        On the probabilities, cross-entropy loss is computed
        Because we are doing multilabel, this is better as the value outtputted (unlike softmax)
        is independent of the other values (while in softmax, probabilities must add up to one)
        '''
            
        if use_focal:
            self.losses = FocalLoss(gamma, alpha)
            print(f'Using Focal Loss: {gamma}, {alpha}')
        else:
            # Regular binary cross entropy
            self.losses = torch.nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        
      
       
        
    def load_checkpoint(self, file_path):
        checkpoint = torch.load(file_path)
        self.model.load_state_dict(checkpoint['state'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        return checkpoint['epoch']

    def save_checkpoint(self, state, filename):
        torch.save(state, filename)      
    
    def evaluate(self, data_loader, is_validation=True):
        
      
        average_loss = 0
        average_auc = 0
        
        
        with torch.no_grad():
            # Go through each batch in the testloader
            for X, y in data_loader:
                input_img = X.cuda(non_blocking=True)
                labels = y.float().cuda(non_blocking=True)
                self.optimizer.zero_grad()
                output = self.model(input_img)
                sig = torch.nn.Sigmoid()
                probabilities = sig(output)
                predictions = probabilities >= 0.30
                # Push data to CPU first
                y = y.cpu()
                predictions = np.array(predictions.cpu()).astype(int)
                probabilities = np.array(probabilities.cpu())
                for i in range(len(y)):
                    if sum(predictions[i,:]) == 0:
                            max_idx = np.where(probabilities[i,:]==max(probabilities[i,:]))
                            predictions[i, max_idx] = 1
                    cur_loss = sk_metrics.hamming_loss(y[i,:], predictions[i,:])
                    auc = sk_metrics.roc_auc_score(y[i,:], probabilities[i,:], average='micro')
          
                   
                    
                    
                    average_loss += cur_loss
                    average_auc += auc
                   
                    
        if is_validation:
            total = len(val_dataset)
        else:
            total = len(test_dataset)
        average_loss = average_loss/total
        average_auc = average_auc/total
        return average_auc, average_loss    
    
        
    def train(self, epochs, trainloader, val_loader=None, checkpoint_path=None, root_path=None):
        # Empty cache before training
        torch.cuda.empty_cache()
        curr_auc = 1000
        if checkpoint_path != None:
            start_epoch = self.load_checkpoint(checkpoint_path)
        else:
            start_epoch = 0
        for e in range(start_epoch, epochs):
            # Get loss per epoch
            running_loss = 0
            for i, (X, y) in enumerate(trainloader):
                input_img = X.cuda(non_blocking=True)
                labels = y.float().cuda(non_blocking=True)
                self.optimizer.zero_grad()
                output = self.model(input_img)
                loss = self.losses(output, labels)
                loss.backward()
                self.optimizer.step()
                running_loss = running_loss + loss.item()
            
            # Stop training if validation auc score is now dropping
            if val_loader != None:
                average_auc, average_loss = self.evaluate(val_loader)
                if average_auc > curr_auc:
                    break
            
            # Find the loss for the current epoch
            loss = running_loss/len(trainloader)
            print(f"Epoch {e + 1} - Loss: {loss}")
          
            state = {
                'epoch': e + 1,
                'state': self.model.state_dict(),
                'optimizer': self.optimizer.state_dict()
            }
            if root_path == None:
                root_path = './checkpoint_epoch_'
            writer.add_scalar("Loss/train", loss, epochs)
            checkpoint_path = os.path.join(root_path + str(e + 1) + '.pth.tar') 
            self.save_checkpoint(state, checkpoint_path)    

In [13]:
from sklearn.metrics import roc_curve, auc
classes = ['Cardiomegaly', 'Emphysema', 'Effusion', 'Hernia',
       'Infiltration', 'Mass', 'Nodule', 'Atelectasis', 'Pneumothorax',
       'Pleural_Thickening', 'Pneumonia', 'Fibrosis', 'Edema',
       'Consolidation']
def plot_images(img, predictions, correctlabel):
        plt.figure(num=None, figsize=(8, 6))
        plt.imshow(img[0,:,:], cmap='gray')
        correct_labels = [classes[idx] for (idx, val) in enumerate(correctlabel) if val == 1]
        
        preds = [classes[idx] for (idx, val) in enumerate(predictions) if val == True]
        correct_labels = ", ".join(correct_labels)
        preds = ", ".join(preds)
        print(f'Correct Labels: {correct_labels}')
        print(f'Predicted Labels: {preds}')
        plt.show()


to_plot=70
def get_images(cur_model):
    cor_count = 0
    incor_count = 0
    with torch.no_grad():
        # Go through each batch in the testloader
        for X, y in test_loader:
            input_img = X.cuda(non_blocking=True)
            labels = y.float().cuda(non_blocking=True)
            cur_model.optimizer.zero_grad()
            output = cur_model.model(input_img)
            sig = torch.nn.Sigmoid()
            probabilities = sig(output)
            predictions = probabilities >= 0.30
            y = y.cpu()
            predictions = np.array(predictions.cpu()).astype(int)
            probabilities = np.array(probabilities.cpu())
            for i in range(len(y)): 
                if sum(predictions[i,:]) == 0:
                    max_idx = np.where(probabilities[i,:]==max(probabilities[i,:]))
                    predictions[i, max_idx] = 1
                    
                cur_loss = sk_metrics.hamming_loss(y[i,:], predictions[i,:])
                if cur_loss == 0.0 and cor_count < to_plot:
                    plot_images(input_img[i].cpu(), predictions[i,:], y[i,:])
                    cor_count += 1
                elif cur_loss > 0.0 and incor_count < to_plot:
                    plot_images(input_img[i].cpu(), predictions[i,:], y[i,:])
                    incor_count += 1
        
        
       



In [14]:
n_classes = 14
learning_rate = 0.0005
chx_model_focal = CHXModel()
chx_model_focal.set_up_model(n_classes, learning_rate, unfreeze=True, pretrained=True, use_focal=True, gamma=2.5, alpha=0.5)
path_to_model = '../input/checkpoint115/checkpoint_epoch_15.pth.tar'
chx_model_focal.train(15, train_loader, root_path='./checkpoint_epoch_', checkpoint_path=path_to_model)

In [15]:
model_auc, model_loss= chx_model_focal.evaluate(test_loader, is_validation=False)
print(f'Average AUC: {model_auc}')
print(f'Average Hamming Loss: {model_loss}')

In [16]:
get_images(chx_model_focal)

<a href="./checkpoint_epoch_15.pth.tar"> Download File </a>